In [1]:
# imports
from keras.preprocessing.image import ImageDataGenerator, array_to_img,img_to_array,load_img
import tensorflow.keras as K
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import glob

In [2]:
# get directories
main_direc = os.getcwd()
images_dir = os.path.join(main_direc, 'data/images/images')

# csv location
artist_csv_loc = os.path.join(main_direc, 'data/artists.csv')

## Loading Data
The below section loads in the data, making necessary preprocess changes.
Generators are used to augment the data.

### Hyperparameter Section for data preprocessing
The following code block allows you to set different hyperparams for loading in
the images.

In [3]:
"""
Set hyperparams for the number of classes and image generators
"""

IMG_WIDTH = 299
IMG_HEIGHT = 299
BATCH_SIZE = 64
NUM_ARTISTS = 10 # this is 11 to get to 10 classes, can be changed...

### DataFrame management
The following code block loads the artists csv into pandas dataframe, sorts by
number of paintings, and makes a dataframe with the top 10 artists by painting
count, to give us the most amount of data possible.

In [4]:
# Collecting Needed Images
artists = pd.read_csv(artist_csv_loc)

# Creating a dataframe with the top 10 artists by number of paintings
artists_sort = artists.sort_values(by=['paintings'], ascending=False)

artists_top = artists_sort.head(NUM_ARTISTS)
print(artists_top)

# Images
artists_dir = os.listdir(images_dir) # Files are named after each artists

# Images DataFrame
artists_top_name = artists_top['name'].str.replace(' ', '_').values

images_df = pd.DataFrame()
for name in artists_top_name:
    images_df = pd.concat([images_df, pd.DataFrame(data={'Path': glob.glob('data/images/images/' + name + '/*'), 'Name': name})], ignore_index=True)

print(images_df)

train_df = images_df.sample(frac=0.8, random_state=200)
test_df = images_df.drop(train_df.index)

if K.backend.image_data_format() == 'channels_first':
    input_shape = (3, IMG_WIDTH, IMG_HEIGHT)
else:
    input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)

    id                   name        years                         genre  \
8    8       Vincent van Gogh  1853 – 1890            Post-Impressionism   
30  30            Edgar Degas  1834 - 1917                 Impressionism   
13  13          Pablo Picasso  1881 - 1973                        Cubism   
15  15  Pierre-Auguste Renoir  1841 - 1919                 Impressionism   
19  19         Albrecht Dürer  1471 - 1528          Northern Renaissance   
46  46           Paul Gauguin  1848 – 1903  Symbolism,Post-Impressionism   
16  16         Francisco Goya  1746 - 1828                   Romanticism   
31  31              Rembrandt  1606 - 1669                       Baroque   
20  20          Alfred Sisley  1839 - 1899                 Impressionism   
32  32                 Titian  1488 - 1576    High Renaissance,Mannerism   

       nationality                                                bio  \
8            Dutch  Vincent Willem van Gogh (Dutch: [ˈvɪnsɛnt ˈʋɪl...   
30          Frenc

### Building generators
The next code block builds generators for augmenting the data.

In [5]:
"""
Build generators
"""

train_generator = ImageDataGenerator(rescale=1.0 / 255,
                                    rotation_range=20,
                                    zoom_range=0.05,
                                    width_shift_range=0.05,
                                    height_shift_range=0.05,
                                    shear_range=0.05,
                                    horizontal_flip=True,
                                    fill_mode="nearest",
                                    validation_split=0.15,
                                    preprocessing_function=preprocess_input
                                    )

test_generator = ImageDataGenerator(rescale=1.0 / 255, preprocessing_function=preprocess_input)

train_gen = train_generator.flow_from_dataframe(
    train_df,
    shuffle=True,
    x_col='Path',
    y_col='Name',
    class_mode='categorical',
    subset="training",
    batch_size=BATCH_SIZE,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    seed=42
)

valid_gen = train_generator.flow_from_dataframe(
    train_df,
    subset="validation",
    shuffle=True,
    x_col='Path',
    y_col='Name',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    seed=42
)

test_gen = test_generator.flow_from_dataframe(
    test_df,
    x_col='Path',
    batch_size=1,
    shuffle=False,
    class_mode=None,
    target_size=(IMG_WIDTH, IMG_HEIGHT)
)

# Set the amount of steps for training, validation, and testing data
# based on the batch size
steps_train = train_gen.n//train_gen.batch_size
steps_valid = valid_gen.n//valid_gen.batch_size
steps_test = test_gen.n//test_gen.batch_size

Found 2165 validated image filenames belonging to 9 classes.
Found 381 validated image filenames belonging to 9 classes.
Found 637 validated image filenames.


## BASELINE MODEL
The following model is a very simple convolutional neural network. It is not
very accurate.

In [ ]:
# number of epochs for the baseline model
n_epochs = 3

sequential_model = tf.keras.Sequential([
  tf.keras.layers.Rescaling(1./255),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(NUM_ARTISTS - 1)
])

# compile model
sequential_model.compile(
  optimizer='adam',
  loss=tf.losses.CategoricalCrossentropy(from_logits=True),
  metrics=['accuracy']
)


In [ ]:
# fit model
# FITTING THE MODEL
sequential_model.fit_generator(
    generator = train_gen,
    steps_per_epoch=steps_train,
    validation_data = valid_gen,
    validation_steps = steps_valid,
    verbose=1,
    epochs=n_epochs
)


## Clay's Model
The following model was designed by Clay Kaufmann. It uses Inception V3 as a
base line, and makes modifications from there.

### Hyperparameter Block
Set different hyperparameters for the model with the following block.

In [9]:
"""
Hyperparameters here:
"""
N_EPOCHS = 10
LEARNING_RATE = 0.001 # 0.001 is the default for Adam set by TensorFlow
OPTIMIZER = tf.optimizers.Adam(learning_rate=LEARNING_RATE)
LOSS_FUNCTION = tf.losses.CategoricalCrossentropy(from_logits=False)

In [10]:
# set the input for VGG
inp = Input(shape=(IMG_HEIGHT,IMG_WIDTH,3))

# load model
base_model = InceptionV3(include_top=False, input_tensor=inp, pooling='max', weights='imagenet')

# set base model to not be trainable
base_model.trainable = False

final_model = tf.keras.Sequential()
final_model.add(base_model)

final_model.add(K.layers.Flatten())
final_model.add(K.layers.BatchNormalization())
final_model.add(Dense(1024, activation='relu'))
final_model.add(K.layers.Dropout(0.6))
final_model.add(K.layers.BatchNormalization())
final_model.add(Dense(512, activation='relu'))
final_model.add(Dense(32, activation='relu'))
final_model.add(K.layers.Dropout(0.6))
final_model.add(Dense(NUM_ARTISTS - 1, activation='softmax'))

final_model.summary()

# compile model
final_model.compile(
  optimizer=OPTIMIZER,
  loss=LOSS_FUNCTION,
  metrics=['accuracy']
)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 2048)              21802784  
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)              0         
_________________________________________________________________
batch_normalization_190 (Bat (None, 2048)              8192      
_________________________________________________________________
dense_4 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
batch_normalization_191 (Bat (None, 1024)              4096      
_________________________________________________________________
dense_5 (Dense)              (None, 512)              

In [11]:
# Fit the model
final_model.fit_generator(
    generator = train_gen,
    steps_per_epoch=steps_train,
    validation_data = valid_gen,
    validation_steps = steps_valid,
    verbose=1,
    epochs=N_EPOCHS
)

Epoch 1/10
33/33 [==============================] - 151s 4s/step - loss: 2.1564 - accuracy: 0.2703 - val_loss: 2.0257 - val_accuracy: 0.2344
Epoch 2/10
33/33 [==============================] - 159s 5s/step - loss: 1.9116 - accuracy: 0.3284 - val_loss: 1.9409 - val_accuracy: 0.3781
Epoch 3/10
33/33 [==============================] - 165s 5s/step - loss: 1.8390 - accuracy: 0.3603 - val_loss: 1.9002 - val_accuracy: 0.2969
Epoch 4/10
33/33 [==============================] - 167s 5s/step - loss: 1.7387 - accuracy: 0.3936 - val_loss: 1.7032 - val_accuracy: 0.4437
Epoch 5/10
33/33 [==============================] - 173s 5s/step - loss: 1.7291 - accuracy: 0.4041 - val_loss: 1.6988 - val_accuracy: 0.4844
Epoch 6/10
33/33 [==============================] - 179s 5s/step - loss: 1.7081 - accuracy: 0.3955 - val_loss: 1.5650 - val_accuracy: 0.5312
Epoch 7/10
33/33 [==============================] - 162s 5s/step - loss: 1.6326 - accuracy: 0.4398 - val_loss: 1.4979 - val_accuracy: 0.5469
Epoch 8/10
33